In [1]:
import requests, json
import pandas as pd
from config import census_key, gkey
from census import Census

ModuleNotFoundError: No module named 'config'

# Building and cleaning neighborhood dataframe

In [2]:
df = pd.read_csv('data_resources/Neighborhoods__Regions_ (1).csv')

In [3]:
df

,OBJECTID,NAME,COMMPLAN,SHARED,COALIT,HORZ_VERT,MAPLABEL,ID,Shape_Length,Shape_Area
0,1,CATHEDRAL PARK,,,NPNS,HORZ,Cathedral Park,31,11434.254777,5.424298e+06
1,2,UNIVERSITY PARK,,,NPNS,HORZ,University Park,88,11950.859827,6.981457e+06
2,3,PIEDMONT,ALBINA,,NPNS,VERT,Piedmont,70,10849.327392,6.079530e+06
3,4,WOODLAWN,ALBINA,,NECN,HORZ,Woodlawn,93,8078.360994,3.870554e+06
4,5,CULLY ASSOCIATION OF NEIGHBORS,,,CNN,HORZ,Cully Association of Neighbors,23,18179.392090,1.658062e+07
...,...,...,...,...,...,...,...,...,...,...
93,94,SUNDERLAND ASSOCIATION OF NEIGHBORS,,,CNN,HORZ,Sunderland Association of Neighbors,85,20706.496916,9.322958e+06
94,95,PORTSMOUTH,,,NPNS,HORZ,Portsmouth,18,10003.795565,5.722844e+06
95,96,HAYDEN ISLAND NEIGHBORHOOD NETWORK,,,NPNS,HORZ,Hayden Island Neighborhood Network,35,27375.215778,1.579258e+07
96,97,BRIDGETON,,,NPNS,HORZ,Bridgeton,11,8635.720662,1.433719e+06


In [4]:
url = 'https://opendata.arcgis.com/datasets/9f50a605cf4945259b983fa35c993fe9_125.geojson'
geo = requests.get(url).json()

In [5]:
names = []
lats = []
lngs = []
for i in geo['features']:
    names.append(i['properties']['NAME'])
    lats.append(i['geometry']['coordinates'][0][0][1])
    lngs.append(i['geometry']['coordinates'][0][0][0])
query_df = pd.DataFrame()
query_df['name'] = names
query_df['lat'] = lats
query_df['lng'] = lngs

pd.set_option('display.max_rows',None)
query_df = query_df.loc[query_df['name'] != 'MC UNCLAIMED #13', :]
query_df = query_df.loc[query_df['name'] != 'CRESTWOOD', :]


In [6]:
maps_base_url = 'https://maps.googleapis.com/maps/api/geocode/json?'
zipcodes = []
for index, row in query_df.iterrows():
    lat = row.lat
    lng = row.lng
    name = row.name
    
    response = requests.get(f"{maps_base_url}latlng={lat},{lng}&key={gkey}").json()
    
    if response['results'][0]['address_components'][-1]['types'][0] == 'postal_code':
        zipcodes.append(response['results'][0]['address_components'][-1]['long_name'])
    elif response['results'][0]['address_components'][-2]['types'][0] == 'postal_code':
        zipcodes.append(response['results'][0]['address_components'][-2]['long_name'])
    else:
        zipcodes.append(response['results'][0]['address_components'][-3]['long_name'])



In [7]:
query_df['Zipcode'] = zipcodes
query_df = query_df.loc[query_df['Zipcode'] != 'Multnomah County', :]
query_df['name'] = [i.title() for i in query_df['name']]

In [8]:
query_df.iloc[4,0] = 'Cully'
query_df.iloc[9,0] = 'Sumner'
query_df.iloc[15,0] = 'Parkrose'
query_df.iloc[24,0] = 'Old Town Chinatown'
query_df.iloc[26,0] = 'Buckman'
query_df.iloc[33,0] = 'Brooklyn'
query_df.iloc[38,0] = 'Sellwood-Moreland'
query_df.iloc[56,0] = 'Southwest Hills'
query_df.iloc[64,0] = 'Centennial'
query_df.iloc[69,0] = 'Wilkes'
query_df.iloc[76,0] = 'Irvington'
query_df.iloc[77,0] = 'Sabin'
query_df.iloc[82,0] = 'Lloyd District'
query_df.iloc[83,0] = 'Goose Hollow'
query_df.iloc[86,0] = 'Hosford-Abernethy'
query_df.iloc[90,0] = 'Northwest District'

query_df

,name,lat,lng,Zipcode
0,Cathedral Park,45.5764,-122.748,97210
1,University Park,45.575,-122.708,97217
2,Piedmont,45.5866,-122.675,97217
3,Woodlawn,45.5663,-122.644,97211
4,Cully,45.5483,-122.615,97213
5,Arbor Lodge,45.5772,-122.679,97217
6,Overlook,45.5546,-122.679,97217
7,Concordia,45.5554,-122.642,97211
8,Parkrose,45.5554,-122.568,97220
9,Sumner,45.5554,-122.568,97220


# Building and cleaning walk score dataframe

In [9]:
walkscore_df = pd.read_csv('data_resources/PDX_Neighborhood_Walkability.csv')

In [10]:
walkscore_df

,Rank,Name,Walk Score,Transit Score,Bike Score,Population
0,1,Pearl,97,89,98,"5,989"
1,2,Old Town Chinatown,97,95,97,"3,873"
2,3,Downtown,96,94,90,"12,803"
3,4,Hollywood,94,70,94,"1,274"
4,5,Goose Hollow,93,84,80,"5,825"
5,6,Kerns,93,65,97,"5,336"
6,7,Northwest District,93,67,89,"12,700"
7,8,Boise,92,55,99,"3,309"
8,9,Hosford-Abernethy,91,60,97,"7,314"
9,10,Buckman,91,61,95,"8,472"


In [11]:
merged_df = query_df.merge(walkscore_df, left_on='name', right_on='Name')

In [12]:
merged_df.drop(columns=['Rank', 'Name', 'Transit Score', 'Bike Score', 'Population'], inplace=True)

In [13]:
merged_df

,name,lat,lng,Zipcode,Walk Score
0,Cathedral Park,45.5764,-122.748,97210,79
1,University Park,45.575,-122.708,97217,53
2,Piedmont,45.5866,-122.675,97217,72
3,Woodlawn,45.5663,-122.644,97211,74
4,Cully,45.5483,-122.615,97213,57
5,Arbor Lodge,45.5772,-122.679,97217,77
6,Overlook,45.5546,-122.679,97217,73
7,Concordia,45.5554,-122.642,97211,76
8,Parkrose,45.5554,-122.568,97220,54
9,Parkrose,45.5467,-122.537,97230,54


# 2012 Census data call

In [14]:
c = Census(census_key, year=2012)
median_incomes = []
pops = []
white_pop = []
black_pop = []
bipoc_pop = []

for i in merged_df['Zipcode']:

    census_data = c.acs5.get(("NAME", 'B01003_001E','B02001_002E', 'B02001_003E', 'B06011_001E',
                             ), {'for': f'zip code tabulation area:{i}'})
    pops.append(census_data[0]['B01003_001E'])
    white_pop.append(census_data[0]['B02001_002E'])
    black_pop.append(census_data[0]['B02001_003E'])
    bipoc_pop.append(census_data[0]['B01003_001E'] - census_data[0]['B02001_002E'])
    median_incomes.append(census_data[0]['B06011_001E'])
census_data

[{'NAME': 'ZCTA5 97211',
  'B01003_001E': 30731.0,
  'B02001_002E': 20505.0,
  'B02001_003E': 6720.0,
  'B06011_001E': 29190.0,
  'zip code tabulation area': '97211'}]

In [15]:
merged_df['Median Income 2012'] = median_incomes
merged_df['Population 2012'] = pops
merged_df['Black Population 2012'] = black_pop
merged_df['BIPOC Population 2012'] = bipoc_pop
merged_df['% BIPOC 2012'] = (merged_df['BIPOC Population 2012']/merged_df['Population 2012'] * 100)

In [16]:
merged_df

,name,lat,lng,Zipcode,Walk Score,Median Income 2012,Population 2012,Black Population 2012,BIPOC Population 2012,% BIPOC 2012
0,Cathedral Park,45.5764,-122.748,97210,79,35374.0,10836.0,213.0,1655.0,15.273164
1,University Park,45.575,-122.708,97217,53,29341.0,31170.0,3655.0,8448.0,27.102984
2,Piedmont,45.5866,-122.675,97217,72,29341.0,31170.0,3655.0,8448.0,27.102984
3,Woodlawn,45.5663,-122.644,97211,74,29190.0,30731.0,6720.0,10226.0,33.275845
4,Cully,45.5483,-122.615,97213,57,31161.0,29940.0,1315.0,4864.0,16.245825
5,Arbor Lodge,45.5772,-122.679,97217,77,29341.0,31170.0,3655.0,8448.0,27.102984
6,Overlook,45.5546,-122.679,97217,73,29341.0,31170.0,3655.0,8448.0,27.102984
7,Concordia,45.5554,-122.642,97211,76,29190.0,30731.0,6720.0,10226.0,33.275845
8,Parkrose,45.5554,-122.568,97220,54,23083.0,29397.0,2691.0,8597.0,29.244481
9,Parkrose,45.5467,-122.537,97230,54,23565.0,41010.0,3651.0,13902.0,33.899049


# 2018 Census data call

In [17]:
c = Census(census_key, year=2018)
median_incomes = []
pops = []
black_pop = []
white_pop = []
bipoc_pop = []

for i in merged_df['Zipcode']:

    census_data = c.acs5.get(("NAME", 'B01003_001E','B02001_002E', 'B02001_003E', 'B06011_001E',
                             ), {'for': f'zip code tabulation area:{i}'})
    pops.append(census_data[0]['B01003_001E'])
    white_pop.append(census_data[0]['B02001_002E'])
    black_pop.append(census_data[0]['B02001_003E'])
    bipoc_pop.append(census_data[0]['B01003_001E'] - census_data[0]['B02001_002E'])
    median_incomes.append(census_data[0]['B06011_001E'])
census_data

[{'NAME': 'ZCTA5 97211',
  'B01003_001E': 34962.0,
  'B02001_002E': 24938.0,
  'B02001_003E': 5673.0,
  'B06011_001E': 36630.0,
  'zip code tabulation area': '97211'}]

In [18]:
merged_df['Median Income 2018'] = median_incomes
merged_df['Population 2018'] = pops
merged_df['Black Population 2018'] = black_pop
merged_df['BIPOC Population 2018'] = bipoc_pop
merged_df['% BIPOC 2018'] = (merged_df['BIPOC Population 2018']/merged_df['Population 2018'] * 100)

In [19]:
zips_group = merged_df.groupby('Zipcode')
zipcode_df = pd.DataFrame(zips_group.mean())
zipcode_df

,Walk Score,Median Income 2012,Population 2012,Black Population 2012,BIPOC Population 2012,% BIPOC 2012,Median Income 2018,Population 2018,Black Population 2018,BIPOC Population 2018,% BIPOC 2018
Zipcode,,,,,,,,,,,
97035,21.000000,40434.0,23820.0,107.0,3073.0,12.900924,48275.0,26014.0,275.0,4322.0,16.614131
97086,22.000000,31017.0,25919.0,163.0,6201.0,23.924534,38206.0,31000.0,592.0,7949.0,25.641935
97201,40.500000,23194.0,14744.0,143.0,2270.0,15.396093,25805.0,18145.0,589.0,4272.0,23.543676
97202,66.333333,28996.0,38951.0,838.0,4844.0,12.436138,36132.0,42822.0,631.0,6447.0,15.055345
97203,53.333333,24099.0,30003.0,2909.0,9041.0,30.133653,28363.0,34365.0,2704.0,8041.0,23.398807
97206,61.333333,26285.0,47544.0,941.0,9036.0,19.005553,31065.0,51698.0,983.0,10975.0,21.229061
97209,95.000000,29754.0,14095.0,794.0,2636.0,18.701667,43873.0,17585.0,788.0,2947.0,16.758601
97210,86.000000,35374.0,10836.0,213.0,1655.0,15.273164,52450.0,12059.0,302.0,1483.0,12.297869
97211,68.600000,29190.0,30731.0,6720.0,10226.0,33.275845,36630.0,34962.0,5673.0,10024.0,28.671129


In [20]:
zipcode_df.to_csv('data_resources/census_walkscore_by_zip.csv')